<a href="https://colab.research.google.com/github/sleighton2022/datasci266-final-project/blob/main/Datasci_266_baseline_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasci 266 final project - pretrained models - no FT

**Description:**

**Structure of the notebook:**

**Expected results:**





## Setup

In [ ]:
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q evaluate
!pip install -q datasets
!pip install -q pandas
!pip install -q rouge_score
!pip install -q nltk
!pip install bert_score
!pip install sentence-transformers
!pip install -q time
!pip install -q torchinfo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu

In [ ]:
#import evaluate
import torch
my_device = "cuda:0" if torch.cuda.is_available() else "cpu"
my_device

#from transformers import T5Tokenizer, TFT5ForConditionalGeneration
#from transformers import BertTokenizer, BertModel

import evaluate
import numpy as np
import time
from transformers import pipeline
from transformers import Trainer, TrainingArguments

In [ ]:
#from transformers import T5Tokenizer, TFT5ForConditionalGeneration  #tensorflow implementation
from transformers import T5Tokenizer, T5ForConditionalGeneration #torch
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
'''
#!pip install git+https://github.com/sleighton2022/datasci266-final-project.git
!wget https://raw.githubusercontent.com/sleighton2022/datasci266-final-project/main/summary_utils.py
from summary_utils import SummaryEvaluator, DatasetManager, SummaryModel
'''


'\n#!pip install git+https://github.com/sleighton2022/datasci266-final-project.git\n!wget https://raw.githubusercontent.com/sleighton2022/datasci266-final-project/main/summary_utils.py\nfrom summary_utils import SummaryEvaluator, DatasetManager, SummaryModel\n'

In [ ]:
'''
# Load dataset
dataset_manager = DatasetManager(dataset_name="xsum", sample_size=10)
sampled_dataset = dataset_manager.load_sampled_dataset()
'''

'\n# Load dataset\ndataset_manager = DatasetManager(dataset_name="xsum", sample_size=10)\nsampled_dataset = dataset_manager.load_sampled_dataset()\n'

In [ ]:
from datasets import load_dataset
xsum_dataset = load_dataset("xsum", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
train_data = xsum_dataset['train']
train_data = train_data.shuffle(seed=42)
test_data = xsum_dataset['test']
test_data = test_data.shuffle(seed=42)
valid_data = xsum_dataset['validation']
valid_data = test_data.shuffle(seed=42)
print(len(train_data))
print(len(test_data))
print(train_data[0])


204045
11334
{'document': 'In Wales, councils are responsible for funding and overseeing schools.\nBut in England, Mr Osborne\'s plan will mean local authorities will cease to have a role in providing education.\nAcademies are directly funded by central government and head teachers have more freedom over admissions and to change the way the school works.\nIt is a significant development in the continued divergence of schools systems on either side of Offa\'s Dyke.\nAnd although the Welsh Government will get extra cash to match the money for English schools to extend the school day, it can spend it on any devolved policy area.\nMinisters have no plans to follow suit.\nAt the moment, governing bodies are responsible for setting school hours and they need ministerial permission to make significant changes.\nThere are already more than 2,000 secondary academies in England and its extension to all state schools is unlikely to shake the Welsh Government\'s attachment to what they call a "com

In [ ]:
#set test dataset for perf eval
test_dataset = test_data.select(range(200))

In [ ]:
#global params
SUMMARY_MAX_LEN = 150
SUMMARY_MIN_LEN = 30

#very important to put in front of each article
PROMPT = 'summarize: '

In [ ]:
#evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")

from sentence_transformers import SentenceTransformer, util
sentence_transformer = SentenceTransformer("all-mpnet-base-v2")
def calculate_vector_similarity(reference_summaries, generated_summaries):
    """
    Calculates cosine similarity between reference and generated summary embeddings.
    """
    ref_embeddings = sentence_transformer.encode(reference_summaries, convert_to_tensor=True)
    gen_embeddings = sentence_transformer.encode(generated_summaries, convert_to_tensor=True)
    cosine_scores = util.cos_sim(ref_embeddings, gen_embeddings)

    avg_similarity = cosine_scores.diagonal().mean().item()  # Average cosine similarity
    return avg_similarity

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Run pretrained models without FT

In [ ]:
# Load the T5 tokenizer and model
model_checkpoints =  ["t5-small",  "t5-base", "t5-large", "facebook/bart-large-cnn", "facebook/bart-large-xsum"]
model_name = model_checkpoints[1]

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
'''
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)
'''
#embedding_layer = model.shared #avilable for T5 models
max_tok_len = 150 #embedding_layer.weight.shape[1]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
model = model.to(my_device)

generated_summaries = []
for article in test_dataset['document']:
    inputs = tokenizer(PROMPT + article, max_length= SUMMARY_MAX_LEN, truncation=True, return_tensors="pt")
    inputs = inputs.to(my_device)
    # Generate Summary
    summary_ids = model.generate(inputs["input_ids"],
                                   min_length= SUMMARY_MIN_LEN,
                                   max_length= SUMMARY_MAX_LEN) #,num_beams=4 , max_new_tokens = max_tok_len,
    candidate = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    generated_summaries.append(candidate)


In [ ]:
#check generated summary vs. reference
for i in range(len(test_dataset[:3])):

  print(test_dataset['summary'][i])
  print(generated_summaries[i])
  print()

A woman who was seriously hurt in a fatal hen party motorway crash is now helping other major trauma victims rebuild their lives.
minibus driver jailed for six years for causing friend's death . 21 women were heading to hen party when minibus was hit by lorry . minibus driver's friend, 18, was killed in the crash .

A Tudor manor house has reopened following a £2.2m makeover.
bramall hall in stockport, gm, has been restored after a 2014 renovation . 1,400 tickets have been sold out for the opening weekend of the event . the manor dates back to the reign of William the Conqueror .

Walt Disney World has unveiled a lighthouse memorial for a young boy who was killed by an alligator while on holiday at the Florida theme park.
two-year-old boy was dragged underwater by an alligator . his parents and sister had been visiting the resort in 2016 . wildlife officials classified the killing as a predatory attack .



In [ ]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"total number of parameters in the model: {total_params}")

total number of parameters in the model: 222903552


In [ ]:
#Print performance
references = test_dataset['summary']
candidates = generated_summaries

rouge_outputs = rouge.compute(predictions=candidates, references=references)
print(rouge_outputs['rouge1'], rouge_outputs['rouge2'], rouge_outputs['rougeL'])
bleu_outputs = bleu.compute(predictions=candidates, references=references)
print(bleu_outputs['bleu'])
bert_outputs = bertscore.compute(predictions=candidates, references=references, lang="en")
print(np.mean(bert_outputs['f1']))
print(calculate_vector_similarity(references, candidates))
print()

0.18188465539614695 0.022947052115021786 0.1279343012197626
0.005988010635235443


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.8493533194065094
0.49085909128189087



In [18]:
test_dataset[2]['document']

'Two-year-old Lane Thomas Graves had been playing in the sand near the resort\'s Seven Seas Lagoon when he was dragged underwater by the creature.\nHis parents and older sister had been visiting the Grand Floridian resort in June 2016 from the state of Nebraska.\nThe lighthouse has been installed near to where the attack occurred.\nWildlife officials classified the killing as a predatory attack, saying the boy did nothing to provoke the alligator.\n"He was in the water not more than ankle deep," the Florida Fish and Wildlife Conservation Commission said in a report, describing how the boy had been gathering water for a sandcastle.\nHis father, Matt Graves, jumped in the water to try to pry open the creature\'s mouth, but "the alligator thrashed and broke Matt\'s grasp and went under the water," according to the report.\nA Disney spokesperson said they hoped the monument would spread awareness for the Lane Thomas Foundation, which also uses the lighthouse as its logo.\nWho is liable for